# Proyecto 1 (Etapa 2) - Generación de Pipelines

En este caso empezaremos mostrando lo que se realizó en el proyecto 1, dejando en claro que usaremos los modelos ya hechos en la etapa anterior para ser automatizados mediante pipelines. Así que después utilizaremos un modelo de TfidVectorizer para el manejo del texto y empezaremos con la generación del pipelines para el proyecto 1 parte 2

### Proyecto 1 (Etapa 1) - Elegibilidad

#### Determinar la elegibilidad de un paciente para ensayos clínicos de cáncer a partir de texto descriptivo
  * Jairo Cespedes - 201912008
  * Daniel Alejandro Ángel Fuertes - 201911345



## Modelos propuestos

 * Árboles de Decisión (Hecho por ambos)
 * Random Forest 
 * Regresión logística

# 1. Importación de librerías 

Para la realización de modelos y limpieza de datos

In [48]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import string
import re

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib as mplt
from sklearn.feature_extraction.text import CountVectorizer
#Regresión logística
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [49]:
# Se cargan los datos. 
data=pd.read_csv('clinical_trials_on_cancer_data_clasificacion.csv', sep=',', encoding = 'utf-8')

In [50]:
data.isnull().sum()

label                  0
study_and_condition    0
dtype: int64

In [51]:
# sacando valores nulos para evitar errores
data.dropna(inplace = True)

data.head()
  

,label,study_and_condition
0,__label__0,study interventions are Saracatinib . recurren...
1,__label__1,study interventions are Stem cell transplantat...
2,__label__0,study interventions are Lenograstim . recurren...
3,__label__0,study interventions are Doxorubicin . stage ii...
4,__label__1,study interventions are Poly I-C . prostate ca...


In [52]:
#Para dejar el label como número binario 0 o 1
data.loc[(data['label'] == '__label__0'), 'label'] = 0
data.loc[(data['label'] == '__label__1'), 'label'] = 1

data.head()

,label,study_and_condition
0,0,study interventions are Saracatinib . recurren...
1,1,study interventions are Stem cell transplantat...
2,0,study interventions are Lenograstim . recurren...
3,0,study interventions are Doxorubicin . stage ii...
4,1,study interventions are Poly I-C . prostate ca...


In [53]:
# Función tomada y adaptada de: https://www.cienciadedatos.net/documentos/py25-text-mining-python.html
def limpiar(texto):
    '''
    Esta función limpia el texto de diferentes palabras y caracteres que puedan no afectar el modelo
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''
    
    # Se convierte todo el texto a minúsculas
    nuevo_texto = texto.lower()
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de conjunciones y palabras innecesarias
    conj = [" and "," are "," for "," of "," with "," to "," as "," or "," do "," the "," in "," than ", "study ", " interventions "]
    for i in conj:
        nuevo_texto = re.sub(i, ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)

    return(nuevo_texto)

In [54]:
#Se utiliza la función para limpiar el texto
data["study_and_condition"]= data['study_and_condition'].apply(lambda text: limpiar(text))
data.head()

,label,study_and_condition
0,0,saracatinib recurrent verrucous carcinoma lar...
1,1,stem cell transplantation hodgkin lymphoma di...
2,0,lenograstim recurrent adult diffuse mixed cel...
3,0,doxorubicin stage iii diffuse large cell lymp...
4,1,poly i c prostate cancer diagnosis unresolved...


In [55]:
#Se pasa la columna label a valores numéricos
data["label"] = data["label"].astype('float64')

# 2. Modelo de TfidVectorizer (Cambiado ya que para hacer el pipeline no es muy recomendable usar bag of words)

Se utliza un modelo de TfidVectorizer para poder crear una matriz de TF-IDF features. para las palabras dentro del dataframe y así cambiarlo a una representación numérica para poder utilizar modelos de clasificación en él

In [56]:
vectorizer = TfidfVectorizer(lowercase=False)
#Se utiliza el modelo en los textos dados
label = vectorizer.fit_transform(data["study_and_condition"])
#print(vectorizer.get_feature_names())

# 3. Construcción del modelo - árbol de decisión

Después de limpiar y discriminar los datos, se procede a crear el árbol de decisión (para generar puntos de decisión de clasificación). 

In [57]:
# Se selecciona la variable objetivo, en este caso "label".
Y = data['label']
# label of words
X = label.toarray()


In [58]:
# Dividir los datos en entrenamiento y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [59]:
# Crear el objeto de arbol de decisión. Utilicemos como criterio de pureza la entropía.
arbol = DecisionTreeClassifier(criterion='entropy', random_state=0)

In [60]:
# Entrenar el modelo de arbol de decisión con los datos de entrenamiento
arbol = arbol.fit(X_train,Y_train)

Con el árbol obtenido calculamos la exactitud sobre el entrenamiento y sobre el test

In [61]:
# Obtener el mejor modelo.
#arbol_final = mejor_modelo.best_estimator_
arbol_final= arbol
# Probemos ahora este modelo sobre test.
y_pred_train = arbol_final.predict(X_train)
y_pred_test = arbol_final.predict(X_test)
print('Exactitud sobre entrenamiento: %.2f' % accuracy_score(Y_train, y_pred_train))
print('Exactitud sobre test: %.2f' % accuracy_score(Y_test, y_pred_test))

Exactitud sobre entrenamiento: 1.00
Exactitud sobre test: 0.78


In [62]:
print(classification_report(Y_test, y_pred_test))

              precision    recall  f1-score   support

         0.0       0.79      0.77      0.78      1228
         1.0       0.77      0.79      0.78      1172

    accuracy                           0.78      2400
   macro avg       0.78      0.78      0.78      2400
weighted avg       0.78      0.78      0.78      2400



# 4. Random Forest Model

Calculamos varios árboles no correlacionados para luego promediarlos y poder obtener los valores deseados, este fue el modelo elegido debido a que es el que mayor accuracy ofrece de los ya mostrados en la etapa 1 (Árbol de decisión, Regresión logística y Random Forest)

In [63]:
#Se importa un modelo de Random Forest
from sklearn.ensemble import RandomForestClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) # 80% de entrenamiento y 20% de test                

In [64]:
#Se crea un clasificador Gaussiano
clf=RandomForestClassifier(n_estimators=100)
#Se hace la prediccion del modelo
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [65]:
# Accuracy del modelo
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8220833333333334


In [66]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.84      0.81      0.82      1228
         1.0       0.80      0.84      0.82      1172

    accuracy                           0.82      2400
   macro avg       0.82      0.82      0.82      2400
weighted avg       0.82      0.82      0.82      2400



# 5. Regresión logística
Se ejecuta el modelo de Regresión Logística y sus respectivas estadísticas

In [67]:
#
# Create an instance of LogisticRegression classifier
#
lr = LogisticRegression(random_state=0, max_iter=1000)
#
# Fit the model
#
lr.fit(X_train, y_train)
#
# Create the predictions
#
y_predict = lr.predict(X_test)
  
# Use metrics.accuracy_score to measure the score
print("LogisticRegression Accuracy %.3f" %metrics.accuracy_score(y_test, y_predict))

LogisticRegression Accuracy 0.807


In [68]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

         0.0       0.82      0.80      0.81      1228
         1.0       0.79      0.82      0.81      1172

    accuracy                           0.81      2400
   macro avg       0.81      0.81      0.81      2400
weighted avg       0.81      0.81      0.81      2400



# 6. Exportar datos limpios
Se exporta el dataframe modificado

In [69]:
feature_names =vectorizer.get_feature_names()
#Se pasa la columna label a valores numéricos
data["label"] = data["label"].astype('int64')
# Create data frame
df=pd.DataFrame(label.toarray(), columns=feature_names)
data.to_csv('DatosModificados.csv', index=False)

# Inicio Etapa 2
En esta sección se realizará el pipeline inspirándose en la limpieza de datos mostrada anteriormente (Con el uso de la función de limpieza de texto, el modelo de TfidVectorizer y los modelos mostrados anteriormente). Todo esto nos permitirá automatizar los modelos junto a su respectiva limpieza y preparación.

In [78]:
# Realizamos el acceso a librerías adicionales para la creación de pipelines
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
import joblib
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from joblib import load
#Se importa un modelo de Random Forest
from sklearn.ensemble import RandomForestClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [80]:
# Se selecciona la variable objetivo, en este caso "label".
Y = data['label']
# label of words
X = data['study_and_condition']
## Dividir los datos en entrenamiento y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


Ya en este momento creamos el pipeline con el TfidVectorizer y los modelos ya desarrollados y ajustados en la etapa 1 del proyecto, además quitamos las palabras que no sirven para el modelo como se explicó anteriormente

In [104]:
print("Creación pipeline")
pipeForest = Pipeline(steps=[                      
                       ('data_preprocessing',TfidfVectorizer(analyzer='word',stop_words=set([" and "," are "," for "," of "," with "," to "," as "," or "," do "," the "," in "," than ", "study ", " interventions "]))),
                       ('rf', clf)
])
pipeRL = Pipeline(steps=[                      
                       ('data_preprocessing',TfidfVectorizer(analyzer='word',stop_words=set([" and "," are "," for "," of "," with "," to "," as "," or "," do "," the "," in "," than ", "study ", " interventions "]))),
                       ('regresion_model', lr)
])
pipeArbol = Pipeline(steps=[                      
                       ('data_preprocessing',TfidfVectorizer(analyzer='word', stop_words=set([" and "," are "," for "," of "," with "," to "," as "," or "," do "," the "," in "," than ", "study ", " interventions "]))),
                       ('arbol_model', arbol)
])

Creación pipeline


In [105]:
#Ajustamos los valores del pipeline a los valores de entrenamiento
pipeForest.fit(X_train,Y_train)
pipeRL.fit(X_train,Y_train)
pipeArbol.fit(X_train,Y_train)
# Guardar nuestra canalización en un archivo pickle binario 
joblib.dump(pipeForest, 'assets/modeloP1E2Forest.pkl')
joblib.dump(pipeRL, 'assets/modeloP1E2Regresion.pkl')
joblib.dump(pipeArbol, 'assets/modeloP1E2Arbol.pkl')

D:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['and', 'are', 'as', 'do', 'for', 'in', 'interventions', 'of', 'or', 'study', 'than', 'the', 'to', 'with'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
D:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['and', 'are', 'as', 'do', 'for', 'in', 'interventions', 'of', 'or', 'study', 'than', 'the', 'to', 'with'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


['assets/modeloP1E2Arbol.pkl']

Finalmente, realizamos la prueba de si funciona el modelo exportado y se calcula su Accuracy

In [106]:
#Cargamos el modelo de Random Forest
modelR = load("assets/modeloP1E2Forest.pkl")
#Se realiza la predicción con el modelo
y_predict = modelR.predict(X_test)
# Usar las métricas para analizar el modelo
print("Model Accuracy %.3f" %metrics.accuracy_score(Y_test, y_predict))
print(classification_report(Y_test, y_predict))

D:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['and', 'are', 'as', 'do', 'for', 'in', 'interventions', 'of', 'or', 'study', 'than', 'the', 'to', 'with'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Model Accuracy 0.830
              precision    recall  f1-score   support

           0       0.85      0.81      0.83      1228
           1       0.81      0.85      0.83      1172

    accuracy                           0.83      2400
   macro avg       0.83      0.83      0.83      2400
weighted avg       0.83      0.83      0.83      2400



In [107]:
#Cargamos el modelo de Regresión Logística
modelRL = load("assets/modeloP1E2Regresion.pkl")
#Se realiza la predicción con el modelo
y_predict = modelRL.predict(X_test)
# Usar las métricas para analizar el modelo
print("Model Accuracy %.3f" %metrics.accuracy_score(Y_test, y_predict))
print(classification_report(Y_test, y_predict))

Model Accuracy 0.825
              precision    recall  f1-score   support

           0       0.84      0.81      0.83      1228
           1       0.81      0.84      0.82      1172

    accuracy                           0.82      2400
   macro avg       0.82      0.82      0.82      2400
weighted avg       0.83      0.82      0.82      2400



In [108]:
#Cargamos el modelo de Arbol de Decisión
modelA = load("assets/modeloP1E2Arbol.pkl")
#Se realiza la predicción con el modelo
y_predict = modelA.predict(X_test)
# Usar las métricas para analizar el modelo
print("Model Accuracy %.3f" %metrics.accuracy_score(Y_test, y_predict))
print(classification_report(Y_test, y_predict))

Model Accuracy 0.767
              precision    recall  f1-score   support

           0       0.78      0.75      0.77      1228
           1       0.75      0.78      0.77      1172

    accuracy                           0.77      2400
   macro avg       0.77      0.77      0.77      2400
weighted avg       0.77      0.77      0.77      2400

